In [1]:
from __future__ import print_function

import os
from multiprocessing import Pool, cpu_count
from nipype.interfaces.ants.segmentation import N4BiasFieldCorrection
import SimpleITK as sitk

In [2]:
def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)


def unwarp_bias_field_correction(arg, **kwarg):
    return bias_field_correction(*arg, **kwarg)


def bias_field_correction(src_path, dst_path):
    print("N4ITK on: ", src_path)
    try:
        n4 = N4BiasFieldCorrection()
        n4.inputs.input_image = src_path
        n4.inputs.output_image = dst_path

        n4.inputs.dimension = 3
        n4.inputs.n_iterations = [100, 100, 60, 40]
        n4.inputs.shrink_factor = 3
        n4.inputs.convergence_threshold = 1e-4
        n4.inputs.bspline_fitting_distance = 300
        n4.run()
    except RuntimeError:
        print("\tFailed on: ", src_path)

    return

In [3]:
parent_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(parent_dir, "final_fyp_prep/data")
data_src_dir = os.path.join(data_dir, "0-raw")
data_dst_dir = os.path.join(data_dir, "3-Bias_field_corrected")
data_labels = ["healthy", "pd"]
create_dir(data_dst_dir)

In [4]:
data_src_paths, data_dst_paths = [], []
for label in data_labels:
    src_label_dir = os.path.join(data_src_dir, label)
    dst_label_dir = os.path.join(data_dst_dir, label)
    create_dir(dst_label_dir)
    for subject in os.listdir(src_label_dir):
        data_src_paths.append(os.path.join(src_label_dir, subject))
        data_dst_paths.append(os.path.join(dst_label_dir, subject))

In [ ]:
# Test
# bias_field_correction(data_src_paths[0], data_dst_paths[0])

# Multi-processing
paras = zip(data_src_paths, data_dst_paths)
pool = Pool(processes=cpu_count())
pool.map(unwarp_bias_field_correction, paras)

N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/healthy/PPMI_106127_MR_3D_T2_SAG_FLAIR_ND_br_raw_20211124065639325_19_S1084532_I1519066.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/pd/PPMI_100889_MR_3D_T2_FLAIR__br_raw_20210916212630170_1_S1063516_I1493043.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/healthy/PPMI_130190_MR_3D_T2_FLAIR__br_raw_20220225014525621_15_S1108182_I1548734.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/pd/PPMI_56744_MR_3D_T2_SAG_FLAIR__br_raw_20210911100444831_189_S1061266_I1490375.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/pd/PPMI_101799_MR_Sag_3D_T2_FLAIR__br_raw_20210924183716464_133_S1066402_I1496494.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/pd/PPMI_133486_MR_3D_T2_FLAIR__br_raw_20220225030037872_22_S1108191_I1548743.nii
N4ITK on:  /home/abdul/Desktop/final_fyp_prep/data/0-raw/pd/PPMI_130013_MR_SAG_3D_FLAIR_br_raw_20220225010932231_97_S1108178_I1548730.nii
N4ITK on: 